In [53]:
import vectorbt as vbt
from tvDatafeed import TvDatafeed, Interval
import pandas as pd
import pandas_ta as ta

In [54]:
tv = TvDatafeed()
price = tv.get_hist("BTCUSD", "INDEX", Interval.in_daily, 1000)

In [56]:
def emaCrossoverStrategy(df, fast, slow):
    df = df.copy()
    df['fast'] = ta.ema(df['close'], length=fast)
    df['slow'] = ta.ema(df['close'], length=slow)
    # df.dropna(inplace=True)
    df["position"] = df['fast'] > df['slow']
    df['position'] = df['position'].astype(int)

    entries = df['position'].diff() > 0
    exits = df['position'].diff() < 0
    return df, entries, exits

In [60]:
df, entries, exits = emaCrossoverStrategy(price, 13, 34)

In [75]:
portfolio = vbt.Portfolio.from_signals(df['close'], entries, exits, init_cash=10000, fees=0.001)
fig = portfolio.plot().update_layout(template='plotly_dark', width=800, height=1000, yaxis_type='log', margin=dict(t=0, b=0, l=50, r=50))
fig.show()

In [83]:
dfStats = pd.DataFrame(portfolio.stats(), columns=["Value"])
dfStats.to_csv("stats.csv")